In [ ]:
!sudo apt install -y libgl1-mesa-glx

In [ ]:
import boto3
import botocore
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

from src.utils import resize_image

# Demo
A notebook to show you a demo of the handwriting recognition. Here we input an image into the prebuilt SageMaker endpoint and the bounding boxes and transcribed text are returned.

*Please note that this network was trained on the [CSafe](https://iastate.figshare.com/articles/CSAFE_Handwriting_Database/10062203) handwriting dataset ([license information](https://creativecommons.org/licenses/by/4.0/)) and it is intended for demostration purposes only.*

In [ ]:
with open("stack_outputs.json") as f:
    sagemaker_configs = json.load(f)

config = botocore.config.Config(read_timeout=200)
runtime = boto3.client('runtime.sagemaker', config=config)

In [ ]:
img = cv2.imread('data/sample/demo.png', cv2.IMREAD_GRAYSCALE)

resized_img, _ = resize_image(img, desired_size=(700, 700))

payload = json.dumps(resized_img.astype(float).tolist())
response = runtime.invoke_endpoint(EndpointName=sagemaker_configs["SageMakerDemoEndpointName"],
                                   ContentType='application/json',
                                   Body=payload)
out = json.loads(response['Body'].read().decode())

bbs = np.array(out["line_bbs"])
decoded = out["decoded"]

In [ ]:
fig, ax = plt.subplots(1, figsize=(20, 15))

for j in range(bbs.shape[0]):
    decoded_string = decoded[j]
    (x, y, w, h) = bbs[j]
    image_h, image_w = resized_img.shape[-2:]
    (x, y, w, h) = (x * image_w, y * image_h, w * image_w, h * image_h)

    ax.text(x+w, y+h/2, decoded_string, bbox=dict(facecolor='red', alpha=0.5))
    rect = patches.Rectangle((x, y), w, h, fill=False, color="r")

    ax.add_patch(rect)
    
ax.set_title("Predicted image")
ax.imshow(resized_img, cmap='Greys_r')
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

# Navigation
Click [here](./1_Introduction.ipynb) to view the introduction of how to label and train your own network